# Load data

Our dataset contains the average GDP per capita of 227 countries from 1970 to 2017 as well as the average of other metrics such as Infant mortality (per 1000 births) and Literacy % for each country

We want our model to be able to predict the GDP per capita for a country based on inputted values for the other metrics.

Here is a link to our dataset: https://www.kaggle.com/code/noxmoon/world-countries-predicting-gdp/input

Here is some more specific information on the columns:

Arable %: Percentage of land capable of cultivating crops
<br>
Crop %: percentage of arable land that is actually under cultivation and used for growing crops
<br>
Other %: percentage of arable land that is not used for crops
<br>
Agriculture: percentage of GDP that comes from agriculture
<br>
Industry: percentage of GDP that comes from industrial activities
<br>
Service: percentage of GDP from the service industry

In [16]:
import pandas as pd

file = 'countries of the world.csv'

df = pd.read_csv(file, decimal = ',')

# Preprocessing for data analysis

In [ ]:
# Fills null values with the average column value by region 

for col in df.columns.values:
  if df[col].isnull().sum() == 0: # if column does not have null values
    continue 
  #otherwise
  med_values = df.groupby('Region')[col].median()
  for region in df['Region'].unique():
    # replace value where value is null and df[Region] = region
    df[col].loc[(df[col].isnull()) & (df['Region'] == region)] = med_values[region] 

In [18]:
# Save preprocessed data before dummy variables to use for data analysis

graph_data = df

# Data Analysis and Graphs

In [19]:
# min column values with country name

for col in df.columns.values[2:]:
  index_of_min = df[col].idxmin()
  min_country = df['Country'].loc[index_of_min]
  min_value = df[col].min()
  print(col + ", " + min_country + ", " + str(min_value))

Population, St Pierre & Miquelon , 7026
Area (sq. mi.), Monaco , 2
Pop. Density (per sq. mi.), Greenland , 0.0
Coastline (coast/area ratio), Afghanistan , 0.0
Net migration, Micronesia, Fed. St. , -20.99
Infant mortality (per 1000 births), Singapore , 2.29
GDP ($ per capita), East Timor , 500.0
Literacy (%), Niger , 17.6
Phones (per 1000), Congo, Dem. Rep. , 0.2
Arable (%), Anguilla , 0.0
Crops (%), Andorra , 0.0
Other (%), Tonga , 33.33
Climate, Afghanistan , 1.0
Birthrate, Hong Kong , 7.29
Deathrate, N. Mariana Islands , 2.29
Agriculture, Singapore , 0.0
Industry, Jersey , 0.02
Service, Equatorial Guinea , 0.062


In [20]:
# max column values with country name

for col in df.columns.values[2:]:
  index_of_max = df[col].idxmax()
  max_country = df['Country'].loc[index_of_max]
  max_value = df[col].max()
  print(col + ", " + max_country + ", " + str(max_value))

Population, China , 1313973713
Area (sq. mi.), Russia , 17075200
Pop. Density (per sq. mi.), Monaco , 16271.5
Coastline (coast/area ratio), Micronesia, Fed. St. , 870.66
Net migration, Afghanistan , 23.06
Infant mortality (per 1000 births), Angola , 191.19
GDP ($ per capita), Luxembourg , 55100.0
Literacy (%), Andorra , 100.0
Phones (per 1000), Monaco , 1035.6
Arable (%), Bangladesh , 62.11
Crops (%), Kiribati , 50.68
Other (%), Anguilla , 100.0
Climate, Armenia , 4.0
Birthrate, Niger , 50.73
Deathrate, Swaziland , 29.74
Agriculture, Liberia , 0.769
Industry, Equatorial Guinea , 0.906
Service, Cayman Islands , 0.954


### Heatmap

Below is a correlation map of all columns in the dataframe 
<br>
Values close to -1 or 1 are a good indicator of important features in the dataset 
<br>
For example, GDP is highly correlated to Phones per 1000 people (0.83)

In [21]:
# Heatmap

import plotly_express as px

corr = graph_data[graph_data.columns.values[2:]].corr() # we only want to take the columns that have numerical data, so we leave out the country and region columns
fig= px.imshow(corr, text_auto=True, width=1300, height=1300)
fig.show()

### Scatterplots

Below is a scatterplot of Phones (per 1000) vs GDP ($ per capita). We can see that the relationship between the two variables is somewhat logarithmic. 

In [31]:
# Scatter Plot

px.scatter(
    graph_data, 
    x="GDP ($ per capita)",
    y="Phones (per 1000)", 
    trendline="ols", 
    hover_data="Country",
    log_x=False,trendline_options=dict(log_x=True))

Below is a scatterplot of GDP ($ per capita) vs Infant mortality (per 1000 births). We can see that the relationship between the two variables is logarithmic. 

In [ ]:
px.scatter(
    graph_data, 
    x="Infant mortality (per 1000 births)",
    y="GDP ($ per capita)", 
    trendline="ols", 
    hover_data="Country",
    log_x=False,trendline_options=dict(log_x=True))

Looking at a Scatter Matrix of our target + a subsection of our features can help us see and understand relationships between variables, especially trhe type of relationship. 

In [ ]:
# Scatter Matrix

px.scatter_matrix(
    graph_data, 
    dimensions=[
        'GDP ($ per capita)',
        'Net migration',
        'Deathrate',
        'Literacy (%)',
        'Phones (per 1000)',
        'Birthrate',
    ], 
    width= 2200, 
    height=1080,
    color='Region', 
    hover_data='Country')

# Birthrate and phones are related logarithmically

# Net migration is clustered around 0, may benefit from quadratic scaling

# other observations

# Preprocessing for Machine Learning

In [ ]:
# Make dummies for Region 

dummies = pd.get_dummies(df['Region'])

df['Asia'] = dummies['ASIA (EX. NEAR EAST)         ']
df['Eastern Europe'] = dummies['EASTERN EUROPE                     ']
df['Northern Africa'] = dummies['NORTHERN AFRICA                    ']
df['Oceania'] = dummies['OCEANIA                            ']
df['Western Europe'] = dummies['WESTERN EUROPE                     ']
df['Sub-Saharan Africa'] = dummies['SUB-SAHARAN AFRICA                 ']
df['Latin America and Caribbean'] = dummies['LATIN AMER. & CARIB    ']
df['Commonwealth of Independent States'] = dummies['C.W. OF IND. STATES ']
df['Near East'] = dummies['NEAR EAST                          ']
df['North America'] = dummies['NORTHERN AMERICA                   ']
df['Baltics'] = dummies['BALTICS                            ']

df.drop('Region', axis=1, inplace=True)

In [ ]:
# Cannot use for prediction

df.drop('Country', axis=1, inplace=True)

In [6]:
# Split data

target = df['GDP ($ per capita)']
features = df.loc[:, df.columns != 'GDP ($ per capita)']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 21) # random state to keep splits the same in different runs

# Decision Trees

### Description of Decision Trees

A decision tree is a ML model that utilize “if-then” logic. The decision nodes (eg “Outlook”) represent a decision that needs to be made or a question that needs to be answered. The branches associated with a decision node represent the possible outcomes of that decision (eg “Sunny”, “Overcast”, “Rainy”). Eventually, the branches lead to leaf nodes which represent the final decision made or outcome. 

![Alt text](https://www.saedsayad.com/images/Decision_tree_r1.png)

In [38]:
# Hyperparameter tuning

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']

max_depth = [10, 20, 30, 40, 50, 60, 70]

min_samples_split = [2, 3, 4, 5, 6]

min_samples_leaf = [1, 2, 3, 4, 5, 6]

param_grid = {
    'criterion' : criterion,
    'max_depth' : max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf' : min_samples_leaf
    
}

dt_grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, n_jobs = -1)
dt_grid_search.fit(x_train, y_train)
print(dt_grid_search.best_params_)

{'criterion': 'friedman_mse', 'max_depth': 60, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [ ]:
# Feature Selection

import numpy as np

my_decision_tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 60, min_samples_split=2, min_samples_leaf=5)

my_decision_tree.fit(x_train, y_train)

feature_importance = my_decision_tree.feature_importances_

my_features = (df.iloc[:, df.columns != 'GDP ($ per capita)'].columns)

def sortSecond(val):
	return val[1]

feature_order = []
importances = [] 

importances = [(my_features[i], feature_importance[i]) for i in range(len(my_features))]
importances.sort(reverse=True, key=sortSecond)

# select the top 10 features from your importances array 
top_dt_features = [feature[0] for feature in importances[:12]]


In [48]:
# Training Optimized Decision Tree Model


dt_features = df[top_dt_features]

x_train_dt, x_test_dt, y_train_dt, y_test_dt = train_test_split(dt_features, target, test_size = 0.2)

my_decision_tree = DecisionTreeRegressor(criterion = 'poisson', max_depth = 50, min_samples_split=5, min_samples_leaf=5)

my_decision_tree.fit(x_train_dt, y_train_dt)

DecisionTreeRegressor(criterion='poisson', max_depth=50, min_samples_leaf=5,
                      min_samples_split=5)

R^2 shows how well the data fit the regression model (the goodness of fit). The value of R^2 can range from 0 to 1, where 0 means that our model does not predict any variability in our data and 1 means that our model prefectly fits/predicts our data.

MAE, or the Mean Absolute Error, is way of seeing how accurate our model is. MAE more intuitive and easy to interpret than R^2. Our Mean Squared Error represents, on average, how much our predicted value differed from the real value.

In [49]:
# Evaluating Decision Tree Model

from sklearn.metrics import r2_score, mean_absolute_error

y_hat_dt = my_decision_tree.predict(x_test_dt)

r2_dt = r2_score(y_test_dt, y_hat_dt)

mae_dt = mean_absolute_error(y_test_dt, y_hat_dt)

print('r2 score: ', r2_dt)
print('mae: ', mae_dt)

r2 score:  0.8390501844149421
mae:  2944.678226363009


# Random Forests

### Description of Random Forests

Random Forests clasify data into smaller classes each with unique values and features, and then trains these subsections using Decision Trees. It starts with data and for every node it asks specific type of question, which will classify given data. After all iterations through tree of questions, random forest is trained and has classified data. 

In [ ]:
# Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

n_estimators = [50, 70, 90, 110, 130, 150]

criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']

max_depth = [10, 20, 30, 40, 50, 60, 70]

min_samples_split = [2, 3, 4, 5]

min_samples_leaf = [1, 2, 3, 4, 5]

param_grid = {
    'n_estimators': n_estimators,
    'criterion' : criterion,
    'max_depth' : max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf' : min_samples_leaf
}

rf_grid_search = GridSearchCV(RandomForestRegressor(), param_grid, n_jobs = -1)
rf_grid_search.fit(x_train, y_train)
print(rf_grid_search.best_params_)

In [ ]:
# Feature Selection

import numpy as np
from sklearn.ensemble import RandomForestRegressor

my_random_forest = RandomForestRegressor(criterion = 'friedman_mse', max_depth = 40, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 90)
my_random_forest.fit(x_train, y_train)

feature_importance = my_random_forest.feature_importances_

my_features = (df.iloc[:, df.columns != 'GDP ($ per capita)'].columns)

def sortSecond(val):
	return val[1]

feature_order = []
importances = []

importances = [(my_features[i], feature_importance[i]) for i in range(len(my_features))]
importances.sort(reverse=True, key=sortSecond)

# select the top 15 features from your importances array
top_names = [feature[0] for feature in importances[:15]]


In [ ]:
# Best Model

rf_features = df[top_names]

x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(rf_features, target, test_size = 0.2)

my_random_forest_2 = RandomForestRegressor(criterion = 'friedman_mse', max_depth = 40, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 90)

my_random_forest_2.fit(x_train_rf, y_train_rf)



In [ ]:
# Evaluate Model

from sklearn.metrics import r2_score, mean_absolute_error

y_hat_rf = my_random_forest_2.predict(x_test_rf)

r2_rf = r2_score(y_test_rf, y_hat_rf)

mae_rf = mean_absolute_error(y_test_rf, y_hat_rf)

print('r2 score: ', r2_rf)
print('mae: ', mae_rf)

# Stochastic Gradient Descent

In [ ]:
# Description of model (how it works) 

#Stochastic gradient decent is an optimization alogrothim used to train machine learning models
#Makes sure that the model doesn't make mistakes such as overfitting
'''
Steps Include:
1) Initialize Parameters --> Random variables for the model's paramters (weights, baises etc)
2) Shuffle Data --> To get rid of baises 
3) Mini-Batch Detection --> Dividng data into small batches for better training
4) Paramter Update --> For each mini-batch, compute gradient of the loss function and update parameters as such
5) Iterate --> Repeat steps 3 & 4 for certain number of iteration (epochs)
'''

In [ ]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
features_scaled = min_max_scaler.fit_transform(features)

x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(features_scaled, target, test_size = 0.2)

In [ ]:
# Hyperparameter tuning

# want loss to be default

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

alpha = [0.001, 0.0001, 0.00001]
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
penalty = ['l1', 'l2', 'elasticnet']

param_grid = {
    'alpha' : alpha,
    'learning_rate' : learning_rate,
    'penalty' : penalty

}

grid_search = GridSearchCV(SGDRegressor(), param_grid, n_jobs = -1)

grid_search.fit(x_train_s, y_train_s)

print(grid_search.best_params_)

In [16]:
# SGD optimized

from sklearn.linear_model import SGDRegressor

my_SGD = SGDRegressor(learning_rate = 'adaptive')

my_SGD.fit(x_train_s, y_train_s)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(learning_rate='adaptive')

In [17]:
y_hat_sgd = my_SGD.predict(x_test_s)

r2_sgd = r2_score(y_test_s, y_hat_sgd)

mae_sgd = mean_absolute_error(y_test_s, y_hat_sgd)

print('r2 score: ', r2_sgd)
print('mae: ', mae_sgd)

r2 score:  0.825185796640268
mae:  2885.8973634957624


# MLP Neural Network


### Description of a MLP NN

An MLP (Multi Layer Perceptron) Neural network works by using different "nodes" in various layers. Many neural networks have three "hidden" layers, and an input and output layer. Each layer will take an input of N features and output some number of features. Each layer must have the same input size as the output of the layer above, or in the case of the input layer, the same number of inputs as features being used. 

Each "node" in a neural network uses an activation function, which determines if the node is activated or disabled. Most commonly and in our case, the ReLU activation function is used. Each node will have a weight attached to it that determines the activation function parameters as well as the activation threshold of the node.

In [18]:
# SKLearn MLPRegressor model was difficult to work with and customize so we moved to pytorch